In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from pymongo import MongoClient
from crud_operations import AnimalShelter

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Grazioso Salvare Dashboard - Gavin Bish'))),
    html.Hr(),
    html.Div([
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain/Wilderness Rescue', 'value': 'Mountain/Wilderness'},
            {'label': 'Disaster/Tracking', 'value': 'Disaster/Tracking'},
            {'label': 'Reset', 'value': 'Reset'}
        ],
        value='Reset',
        inline=True
        )

    ]),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
                         page_size=10,
                         row_selectable="single",
                         sort_action="native",
                         filter_action="native",
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    # Define breed mappings for rescue types
    rescue_type_mapping = {
        'Water Rescue': ["Labrador Retriever", "Golden Retriever", "Newfoundland"],
        'Mountain/Wilderness': ["German Shepherd", "Border Collie", "Bernese Mountain Dog"],
        'Disaster/Tracking': ["Bloodhound", "Beagle", "Australian Shepherd"],
    }

    # Default query
    query = {}

    # Apply filter logic
    if filter_type in rescue_type_mapping:
        query = {"breed": {"$in": rescue_type_mapping[filter_type]}}
    elif filter_type == 'Reset':
        query = {}  # No filter

    # Query MongoDB
    result = db.read(query)

    if not result:
        return []  # Return empty table if no results

    # Convert to DataFrame
    df = pd.DataFrame.from_records(result)

    # Remove '_id' column if present
    if '_id' in df.columns:
        df.drop(columns=['_id'], inplace=True)

    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):
    if viewData is None or len(viewData) == 0:
        return "No data available for visualization."

    # Convert filtered data into a DataFrame
    dff = pd.DataFrame(viewData)

    # Check if the 'breed' column exists in the dataset
    if 'breed' not in dff.columns:
        return "Breed data is not available."

    # Aggregate data: Count occurrences of each breed
    breed_counts = dff['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']  # Rename columns for clarity

    # Plot a bar chart using Plotly
    fig = px.bar(
        breed_counts,
        x='breed',
        y='count',
        title="Breed Distribution",
        labels={'count': 'Number of Animals', 'breed': 'Breed'},
    )

    return [
        dcc.Graph(figure=fig)
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    # Check if selected_columns is None or empty
    if not selected_columns:
        return []  # Return an empty style list if no columns are selected

    # Generate style for selected columns
    return [
        {
            'if': {'column_id': col},
            'background_color': '#D2F3FF'
        }
        for col in selected_columns
    ]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"), Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    # Check if viewData is None or empty
    if not viewData:
        return "No data available to display on the map."

    # Convert viewData to a DataFrame
    dff = pd.DataFrame(viewData)

    # Ensure 'location_lat' and 'location_long' columns exist
    if 'location_lat' not in dff.columns or 'location_long' not in dff.columns:
        return "Location data is missing in the dataset."

    # Ensure the index is valid
    if not index or len(index) == 0 or index[0] >= len(dff):
        return "No valid selection to display on the map."

    # Retrieve the selected row
    row = index[0]
    selected_lat = dff.iloc[row]['location_lat']
    selected_long = dff.iloc[row]['location_long']
    breed = dff.iloc[row]['breed']
    name = dff.iloc[row]['name']

    # Check if latitude and longitude are valid
    if pd.isna(selected_lat) or pd.isna(selected_long):
        return "Selected data does not contain valid location coordinates."

    # Generate the map with the selected data
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[selected_lat, selected_long], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[selected_lat, selected_long], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:12018/
